# HPC Mini-Challenge 2 - Beschleunigung in Data Science
## Teil 2: GPU
#### FHNW - FS2024

Original von S. Suter, angepasst von S. Marcin und M. Stutz

Abgabe von: <font color='blue'>Dominik Filliger</font>

#### Ressourcen
* [Überblick GPU Programmierung](https://www.cherryservers.com/blog/introduction-to-gpu-programming-with-cuda-and-python)
* [CUDA Basic Parts](https://nyu-cds.github.io/python-gpu/02-cuda/)
* [Accelerate Code with CuPy](https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56)
* Vorlesungen und Beispiele aus dem Informatikkurs PAC (parallel computing), siehe Ordner "resources"
* CSCS "High-Performance Computing with Python" Kurs, Tag 3: 
    - JIT Numba GPU 1 + 2
    - https://youtu.be/E4REVbCVxNQ
    - https://github.com/eth-cscs/PythonHPC/tree/master/numba-cuda
    - Siehe auch aktuelles Tutorial von 2021
* [Google CoLab](https://colab.research.google.com/) oder ggf. eigene GPU.


In [ ]:

import os
import glob
import imageio
import logging
import numpy as np
import cupy as cp
import time
from numba import cuda, vectorize, float32
import matplotlib.pyplot as plt

# TODO REMOVE FOR FINAL SUBMISSION
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_mri_images(subfolder='001'):
    """Load MRI images from the specified subfolder"""
    try:
        folders = os.path.join('adni_png', subfolder)
        if not os.path.exists(folders):
            raise FileNotFoundError(f"MRI image folder not found: {folders}")
        
        files = sorted(glob.glob(f"{folders}/*.png"))
        if not files:
            raise FileNotFoundError(f"No MRI images found in {folders}")
        
        images = np.array([imageio.imread(f) for f in files], dtype=np.float32)
        names = [f[-17:-4] for f in files]
        logger.info(f"Successfully loaded {len(images)} MRI images from {subfolder}")
        return images, names
    except Exception as e:
        logger.error(f"Error loading MRI images: {str(e)}")
        raise

In [1]:
# Dummy Beispiel zum testen mit Numba

import math
from numba import vectorize
import numpy as np

@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)
  

a = np.arange(4096,dtype=np.float32)
gpu_sqrt(a)

/home/manuel/miniconda3/envs/hpc/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([ 0.       ,  1.       ,  1.4142135, ..., 63.97656  , 63.98437  ,
       63.992188 ], dtype=float32)

### 5 GPU Rekonstruktion

Implementiere eine SVD-Rekonstruktionsvariante auf der GPU oder in einem hybriden Setting. Code aus dem ersten Teil darf dabei verwendet werden. Wähle  bewusst, welche Teile des Algorithms in einem GPU Kernel implementiert werden und welche effizienter auf der CPU sind. Ziehe dafür Erkenntnisse aus dem ersten Teil mit ein. Es muss mindestens eine Komponente des Algorithmuses in einem GPU-Kernel implementiert werden. Dokumentiere Annahmen, welche du ggf. zur Vereinfachung triffst. Evaluiere, ob du mit CuPy oder Numba arbeiten möchtest.

Links:
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)

In [ ]:
### BEGIN SOLUTION
import math
import numpy as np
import cupy as cp
from numba import cuda
import time
import matplotlib.pyplot as plt
import timeit

# todo 1
def reconstruct_svd_broadcast1(u,s,vt,k):
    ### BEGIN SOLUTION
    reco = u[:,:k] * s[:k] @ vt[:k,:]
    ### END SOLUTION
    return reco

def reconstruct_svd_cp(u, s, vt, k):
    """
    Perform SVD reconstruction using CuPy's built-in dot product.
    """
    return cp.asnumpy(cp.dot(u[:,:k], cp.dot(cp.diag(s[:k]), vt[:k,:])))

def reconstruct_svd_cp_einsum(u, s, vt, k):
    """
    Perform SVD reconstruction using CuPy and einsum for matrix multiplication.
    """
    return cp.asnumpy(cp.einsum('ik,k,kj->ij', u[:,:k], s[:k], vt[:k,:]))

def reconstruct_svd_cp_broadcast(u, s, vt, k):
    """
    CuPy SVD reconstruction using broadcasting for the multiplication of S.
    """
    return cp.asnumpy(cp.dot(u[:,:k], cp.multiply(s[:k].reshape(-1, 1), vt[:k,:])))

@cuda.jit
def reconstruct_svd_kernel(u, s, vt, out, k):
    """
    CUDA kernel that reconstructs a matrix from SVD components:
      out = u * s * vt, up to rank k.
    """
    i, j = cuda.grid(2)
    if i < u.shape[0] and j < vt.shape[1]:
        tmp = 0.0
        for r in range(k):
            tmp += u[i, r] * s[r] * vt[r, j]
        out[i, j] = tmp

def reconstruct_svd_numba(u, s, vt, k):
    """
    Multiply U, S, and V^T on the GPU using a custom CUDA kernel.
    """
    U_k = u[:,:k]
    S_k = s[:k]
    VT_k = vt[:k,:]

    # Prepare output on GPU
    out = np.zeros((U_k.shape[0], VT_k.shape[1]), dtype=np.float32)

    # Launch kernel
    threads_per_block = (16, 16)
    blocks_per_grid = (
        math.ceil(U_k.shape[0] / threads_per_block[0]),
        math.ceil(VT_k.shape[1] / threads_per_block[1])
    )
    reconstruct_svd_kernel[blocks_per_grid, threads_per_block](U_k, S_k, VT_k, out, S_k.shape[0])
    return out

### END SOLUTION

In [ ]:

def measure_metrics(original, reconstructed):
    """
    Compute MSE and PSNR for the given original and reconstructed images.
    """
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:  # Perfect reconstruction
        psnr = float('inf')
    else:
        psnr = round(20 * np.log10(255.0 / np.sqrt(mse)), 5)
    return mse, psnr

def plot_reconstruction_comparison(test_image, k_values=(10, 25, 50, 100)):
    """
    For each method, reconstruct the test_image with various k-values,
    measuring decomposition and reconstruction times separately.
    """
    implementations = {
        'CuPy Basic': reconstruct_svd_cp,
        'CuPy Einsum': reconstruct_svd_cp_einsum,
        'CuPy Broadcast': reconstruct_svd_cp_broadcast,
        'Numba CUDA': reconstruct_svd_numba,
        'CPU Broadcast': reconstruct_svd_broadcast1,
    }

    results = {name: [] for name in implementations}

    # Perform SVD decomposition once
    U, S, VT = np.linalg.svd(test_image, full_matrices=False)

    # Pre-transfer data to GPU for GPU implementations
    U_gpu = cp.asarray(U, dtype=cp.float32)
    S_gpu = cp.asarray(S, dtype=cp.float32)
    VT_gpu = cp.asarray(VT, dtype=cp.float32)

    for k in k_values:
        first_mse = None
        first_psnr = None
        
        for name, func in implementations.items():
            recon_times, mse_list, psnr_list = [], [], []
            
            # Prepare inputs based on implementation type
            if name == 'CPU Broadcast':
                u_input, s_input, vt_input = U, S, VT
            else:
                u_input, s_input, vt_input = U_gpu, S_gpu, VT_gpu
            # warmup
            func(u_input, s_input, vt_input, k)

            recon_time = timeit.timeit(
                lambda: func(u_input, s_input, vt_input, k),
                number=3
            )
            reconstructed = func(u_input, s_input, vt_input, k)
            
            mse, psnr = measure_metrics(test_image, reconstructed)
            
            # Store first MSE/PSNR values to compare against
            if first_mse is None:
                first_mse = mse
                first_psnr = psnr
            else:
                # Assert that MSE and PSNR are equal across implementations
                assert np.allclose(mse, first_mse, rtol=1e-5), f"MSE mismatch for {name}"
                assert np.allclose(psnr, first_psnr, rtol=1e-5), f"PSNR mismatch for {name}"

            results[name].append({
                'k': k,
                'recon_time': recon_time,
                'mse': mse,
                'psnr': psnr,
                'reconstructed': reconstructed
            })

    # Plot reconstruction time, MSE, PSNR
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    for name, data in results.items():
        k_list = [d['k'] for d in data]
        recon_times = [d['recon_time'] for d in data]
        mse_list = [d['mse'] for d in data]
        psnr_list = [d['psnr'] for d in data]

        axes[0].plot(k_list, recon_times, '-o', label=name)
        axes[1].plot(k_list, mse_list, '-o', label=name)
        axes[2].plot(k_list, psnr_list, '-o', label=name)

    axes[0].set_title('Reconstruction Time')
    axes[0].set_xlabel('k')
    axes[0].set_ylabel('Time (s)')
    axes[0].set_yscale('log')

    axes[1].set_title('MSE')
    axes[1].set_xlabel('k')
    axes[1].set_ylabel('Mean Squared Error')
    # log scale
    axes[1].set_yscale('log')

    axes[2].set_title('PSNR')
    axes[2].set_xlabel('k')
    axes[2].set_ylabel('dB')
    # log scale
    axes[2].set_yscale('log')

    for ax in axes:
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()

    # Plot reconstructions and difference maps for k=50
    k_idx = k_values.index(50)  # Get index of k=50 results
    
    # Create figure with subplots
    n_methods = len(implementations)
    fig, axes = plt.subplots(3, n_methods, figsize=(4*n_methods, 12))
    
    # Plot original image in first row, first column
    axes[0,0].imshow(test_image, cmap='gray')
    axes[0,0].set_title('Original Image')
    axes[0,0].axis('off')
    
    for idx, (name, method_results) in enumerate(results.items()):
        reconstructed = method_results[k_idx]['reconstructed']
        diff_map = np.abs(test_image - reconstructed)
        
        # Plot reconstructed image
        if idx > 0:  # Skip first column of first row (used for original)
            axes[0,idx].imshow(reconstructed, cmap='gray')
            axes[0,idx].set_title(f'{name}\nk=50')
            axes[0,idx].axis('off')
        
        # Plot difference map
        im = axes[1,idx].imshow(diff_map, cmap='hot')
        axes[1,idx].set_title(f'Difference Map\nMSE={method_results[k_idx]["mse"]:.2e}')
        axes[1,idx].axis('off')
        plt.colorbar(im, ax=axes[1,idx])
        
        # Plot intensity profile
        middle_row = test_image.shape[0]//2
        axes[2,idx].plot(test_image[middle_row,:], label='Original')
        axes[2,idx].plot(reconstructed[middle_row,:], '--', label='Reconstructed')
        axes[2,idx].set_title('Middle Row Intensity Profile')
        axes[2,idx].legend()
        axes[2,idx].grid(True)
    
    plt.tight_layout()
    plt.show()

images, names = load_mri_images(subfolder='001')
test_image = np.ascontiguousarray(images[0])

plot_reconstruction_comparison(test_image)


<font color='blue'>Antwort hier eingeben</font>

#### 5.2 GPU-Kernel Performance

##### 5.2.1 Blocks und Input-Grösse

Links: 
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)
* [NVIDIA Kapitel zu "Strided Access"](https://spaces.technik.fhnw.ch/multimediathek/file/cuda-best-practices-in-c)
* https://developer.nvidia.com/blog/cublas-strided-batched-matrix-multiply/
* https://developer.nvidia.com/blog/how-access-global-memory-efficiently-cuda-c-kernels/

Führe 2-3 Experimente mit unterschiedlichen Blockkonfigurationen und Grösse der Input-Daten durch. Erstelle dafür ein neues Datenset mit beliebig grossen Matrizen, da die GPU besonders geeignet ist um grosse Inputs zu verarbeiten (Verwende diese untschiedlich grossen Matrizen für alle nachfolgenden Vergeliche und Tasks ebenfalls). Messe die Performance des GPU-Kernels mittels geeigneten Funktionen. Welche Blockgrösse in Abhängigkeit mit der Input-Grösse hat sich bei dir basierend auf deinen Experimenten als am erfolgreichsten erwiesen? Welches sind deiner Meinung nach die Gründe dafür? Wie sind die Performance Unterschiede zwischen deiner CPU und GPU Implementierung? Diskutiere deine Analyse (ggf. mit Grafiken).

In [ ]:
# Matrix sizes to test
matrix_sizes = [128, 512, 1024, 2048, 4096]

# Block configurations to test - powers of 2, respecting max 1024 threads
block_configs = [
    (x, y) for x, y in [
        (2**i, 2**j) 
        for i in range(1,6) 
        for j in range(1,9)
    ] if x * y <= 1024 and x <= y
]

results_basic = []

# Precompute SVD decompositions for each matrix size
svd_components = {}
for N in matrix_sizes:
    A = np.random.randn(N, N).astype(np.float32)
    u, s, vt = cp.linalg.svd(cp.asarray(A), full_matrices=False)
    k = min(u.shape[1], vt.shape[0])
    svd_components[N] = {
        'u': cp.asnumpy(u),
        's': cp.asnumpy(s),
        'vt': cp.asnumpy(vt),
        'k': k
    }


In [ ]:
### BEGIN SOLUTION

for N in matrix_sizes:
    # Get precomputed components
    u = svd_components[N]['u']
    s = svd_components[N]['s'] 
    vt = svd_components[N]['vt']
    k = svd_components[N]['k']

    # CPU baseline
    start_cpu = time.time()
    C_cpu = reconstruct_svd_broadcast1(u, s, vt, k)
    cpu_time = time.time() - start_cpu
    print(f"CPU time: {cpu_time:.4f} seconds")
    
    # Allocate device memory
    u_device = cuda.to_device(u)
    s_device = cuda.to_device(s)
    vt_device = cuda.to_device(vt)
    C_device = cuda.device_array((N, N), dtype=np.float32)

    for block_size in block_configs:
        threadsperblock = block_size
        blockspergrid_x = math.ceil(N / threadsperblock[0])
        blockspergrid_y = math.ceil(N / threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        
        # Warm-up run
        reconstruct_svd_kernel[blockspergrid, threadsperblock](u_device, s_device, vt_device, C_device, k)
        cuda.synchronize()
        
        # Timed run
        start_gpu = time.time()
        reconstruct_svd_kernel[blockspergrid, threadsperblock](u_device, s_device, vt_device, C_device, k)
        gpu_time = time.time() - start_gpu
        cuda.synchronize()
        
        # Copy result back to host
        C_gpu = C_device.copy_to_host()
        
        # Calculate error
        error = np.mean(np.abs(C_cpu - C_gpu))
        speedup = cpu_time / gpu_time
        
        results_basic.append({
            'kernel': 'basic',
            'matrix_size': N,
            'block_size': block_size,
            'cpu_time': cpu_time,
            'gpu_time': gpu_time,
            'speedup': speedup,
        })
        
        print(f"Block size {block_size}: GPU time = {gpu_time:.4f}s, Speedup = {speedup:.2f}x, Error = {error:.2e}")
        
    # Clean up device memory
    del u_device, s_device, vt_device, C_device

### END SOLUTION

In [ ]:
def plot_results(results, kernel_name):
    """
    Plot heatmaps and bar plots for each matrix size showing GPU times and CPU comparison.
    """
    # Extract unique sizes and configurations
    block_sizes = sorted(list(set([tuple(r['block_size']) for r in results])))
    matrix_sizes = sorted(list(set(r['matrix_size'] for r in results)))
    
    # Prepare data structures
    gpu_times = {}
    cpu_times = {}
    
    # Organize data
    for r in results:
        bs = tuple(r['block_size'])
        N = r['matrix_size']
        gpu_times.setdefault(bs, {})[N] = r['gpu_time']
        if N not in cpu_times:
            cpu_times[N] = r['cpu_time']
    
    # Create subplots for each matrix size
    for N in matrix_sizes:
        plt.figure(figsize=(18, 5))
        
        # --- Left subplot: Heatmap
        plt.subplot(1, 2, 1)
        
        # Extract block dimensions for heatmap axes
        block_x = sorted(list(set([bs[0] for bs in block_sizes])))
        block_y = sorted(list(set([bs[1] for bs in block_sizes])))
        
        # Create heatmap data - fill lower triangle
        heatmap_data = np.zeros((len(block_x), len(block_y)))
        for i, bx in enumerate(block_x):
            for j, by in enumerate(block_y):
                if (bx, by) in gpu_times and N in gpu_times[(bx, by)]:
                    heatmap_data[i,j] = gpu_times[(bx, by)][N]
                else:
                    heatmap_data[i,j] = np.nan
        
        # Find minimum time (excluding NaN values)
        min_time = np.nanmin(heatmap_data)
        
        plt.imshow(heatmap_data, cmap='magma_r', aspect='auto')  # Reversed colormap so lowest time is highlighted
        plt.colorbar(label='Time (s)')
        plt.title(f'GPU Times Heatmap - Matrix Size {N}x{N}\nBest time: {min_time:.4f}s')
        plt.xlabel('Block Y Size')
        plt.ylabel('Block X Size')
        plt.xticks(range(len(block_y)), block_y)
        plt.yticks(range(len(block_x)), block_x)
        
        # Highlight the minimum value
        min_idx = np.where(heatmap_data == min_time)
        plt.plot(min_idx[1], min_idx[0], 'r*', markersize=15, label=f'Best: {min_time:.4f}s')
        plt.legend()
        
        # --- Right subplot: Bar plot
        plt.subplot(1, 2, 2)
        
        # Prepare data for bar plot
        gpu_times_N = []
        gpu_labels = []
        
        for bs in block_sizes:
            if N in gpu_times[bs]:
                gpu_times_N.append(gpu_times[bs][N])
                gpu_labels.append(f'Block {bs}')
        
        # Sort by execution time
        sorted_indices = np.argsort(gpu_times_N)
        sorted_times = [gpu_times_N[i] for i in sorted_indices]
        sorted_labels = [gpu_labels[i] for i in sorted_indices]
        
        # Create bar plot
        bars = plt.bar(range(len(sorted_times)), sorted_times)
        plt.xticks(range(len(sorted_times)), sorted_labels, rotation=45, ha='right')
        
        # Add CPU time reference line
        plt.axhline(y=cpu_times[N], color='r', linestyle='--', label=f'CPU Time')
        
        plt.yscale('log')
        plt.ylabel('Time (s)')
        plt.title(f'Execution Times - Matrix Size {N}x{N}')
        plt.grid(True, which='both', ls='-', alpha=0.2)
        plt.legend()
        
        plt.tight_layout()
        plt.show()


In [ ]:
plot_results(results_basic, 'Basic Kernel')

matrix_sizes = sorted(list(set(r['matrix_size'] for r in results_basic)))

for N in matrix_sizes:
    # Filter results for this matrix size
    results_N = [r for r in results_basic if r['matrix_size'] == N]
    best_basic = min(results_N, key=lambda x: x['gpu_time'])
    
    print(f"\n=== Best Configuration for Basic Kernel (Matrix Size {N}x{N}) ===")
    print(f"Block Size: {best_basic['block_size']}")
    print(f"GPU Time: {best_basic['gpu_time']:.4f} seconds") 
    print(f"Speedup over CPU: {best_basic['speedup']:.2f}x")

<font color='blue'>Antwort hier eingeben</font>

##### 5.2.2 Shared Memory auf der GPU
Optimiere deine Implementierung von oben indem du das shared Memory der GPU verwendest. Führe wieder mehrere Experimente mit unterschiedlicher Datengrösse durch und evaluiere den Speedup gegenüber der CPU Implementierung.

Links:
* [Best Practices Memory Optimizations](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)
* [Examples: Matrix Multiplikation und Shared Memory](https://numba.readthedocs.io/en/latest/cuda/examples.html)

In [ ]:
### BEGIN SOLUTION
def get_kernel(threads_per_block):
    @cuda.jit
    def reconstruct_svd_numba_shared_memory(u, s, vt, C, k):
        block_i = cuda.blockIdx.x
        block_j = cuda.blockIdx.y
        thread_i = cuda.threadIdx.x
        thread_j = cuda.threadIdx.y
        i, j = cuda.grid(2)

        tmp = 0.0
        
        # generate shared memory arrays of size threads_per_block x threads_per_block
        u_shared = cuda.shared.array(shape=(threads_per_block[0], threads_per_block[1]), dtype=float32)
        vt_shared = cuda.shared.array(shape=(threads_per_block[0], threads_per_block[1]), dtype=float32)
        s_shared = cuda.shared.array(shape=(threads_per_block[0]), dtype=float32)

        # determine how many blocks we need to load
        num_blocks = math.ceil(min(k, vt.shape[0], u.shape[1]) / threads_per_block[0])
        for m in range(num_blocks):
            u_shared[thread_i, thread_j] = u[block_i * threads_per_block[0] + thread_i, m * threads_per_block[1] + thread_j]
            vt_shared[thread_i, thread_j] = vt[m * threads_per_block[0] + thread_i, block_j * threads_per_block[1] + thread_j]
            if thread_j == 0:
                s_shared[thread_i] = s[m * threads_per_block[0] + thread_i]

            cuda.syncthreads()
            for l in range(threads_per_block[0]):
                if l + m * threads_per_block[0] < k:
                    tmp += u_shared[thread_i, l] * s_shared[l] * vt_shared[l, thread_j]
            cuda.syncthreads()

        C[i, j] = tmp
        
    return reconstruct_svd_numba_shared_memory



# Store results
results_shared = []

for N in matrix_sizes:
    # Get precomputed components
    u = svd_components[N]['u']
    s = svd_components[N]['s'] 
    vt = svd_components[N]['vt']
    k = svd_components[N]['k']

    # CPU baseline
    start_cpu = time.time()
    C_cpu = reconstruct_svd_broadcast1(u, s, vt, k)
    cpu_time = time.time() - start_cpu
    print(f"CPU time: {cpu_time:.4f} seconds")
    
    # Allocate device memory
    u_device = cuda.to_device(u)
    s_device = cuda.to_device(s)
    vt_device = cuda.to_device(vt)
    C_device = cuda.device_array((N, N), dtype=np.float32)

    for block_size in block_configs:
        threadsperblock = block_size
        blockspergrid_x = math.ceil(N / threadsperblock[0])
        blockspergrid_y = math.ceil(N / threadsperblock[1])
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        
        kernel = get_kernel(threadsperblock)
        
        # Warm-up run
        kernel[blockspergrid, threadsperblock](u_device, s_device, vt_device, C_device, k)
        cuda.synchronize()
        
        # Timed run
        start_gpu = time.time()
        kernel[blockspergrid, threadsperblock](u_device, s_device, vt_device, C_device, k)
        gpu_time = time.time() - start_gpu
        cuda.synchronize()
        
        # Copy result back to host
        C_gpu = C_device.copy_to_host()
        
        speedup = cpu_time / gpu_time
        
        results_shared.append({
            'kernel': 'shared',
            'matrix_size': N,
            'block_size': block_size,
            'cpu_time': cpu_time,
            'gpu_time': gpu_time,
            'speedup': speedup,
        })
        
        print(f"Block size {block_size}: GPU time = {gpu_time:.4f}s, Speedup = {speedup:.2f}x")
        
    # Clean up device memory
    del u_device, s_device, vt_device, C_device
### END SOLUTION

In [ ]:
plot_results(results_shared, 'Shared Kernel')

best_shared = min(results_shared, key=lambda x: x['gpu_time'])

for N in matrix_sizes:
    # Filter results for this matrix size
    results_N = [r for r in results_shared if r['matrix_size'] == N]
    best_shared = min(results_N, key=lambda x: x['gpu_time'])
    
    print(f"\n=== Best Configuration for Shared Kernel (Matrix Size {N}x{N}) ===")
    print(f"Block Size: {best_shared['block_size']}")
    print(f"GPU Time: {best_shared['gpu_time']:.4f} seconds") 
    print(f"Speedup over CPU: {best_shared['speedup']:.2f}x")


Was sind deine Erkenntnisse bzgl. GPU-Memory-Allokation und des Daten-Transferes auf die GPU? Interpretiere deine Resultate.

<font color='blue'>Antwort hier eingeben</font>

##### 5.2.3 Bonus: Weitere Optimierungen
Optimiere deine Implementation von oben weiter. Damit du Erfolg hast, muss der Data-Reuse noch grösser sein.

In [ ]:
### BEGIN SOLUTION
def allocate_pinned(shape, dtype=np.float32):
    """
    Allocate pinned (page-locked) memory on the host.
    This can help reduce transfer overhead to/from the GPU.
    """
    return cuda.pinned_array(shape, dtype=dtype)

def get_optimized_kernel(threads_per_block):
    """
    Returns an 'optimized' GPU kernel for partial SVD reconstruction
    that builds on the shared memory approach and uses a bit more
    local register logic to reduce repeated memory loads.
    """
    blockDimX, blockDimY = threads_per_block

    @cuda.jit
    def reconstruct_svd_optimized(u, s, vt, C, k):
        """
        Further optimized reconstruction:
          - We still chunk across k in increments of blockDimX
          - We keep partial sums in registers
          - We rely on shared memory for local tile data
        """
        # If k <= 0, no reconstruction
        if k <= 0:
            return

        # Global (i, j)
        i = cuda.blockIdx.x * blockDimX + cuda.threadIdx.x
        j = cuda.blockIdx.y * blockDimY + cuda.threadIdx.y

        # Boundary checks
        if i >= C.shape[0] or j >= C.shape[1]:
            return

        # Local coords
        local_x = cuda.threadIdx.x
        local_y = cuda.threadIdx.y

        # Prepare shared memory
        # We store slices from U, V^T, and s in a tile of shape (blockDimX, blockDimY), plus s in 1D
        tileU = cuda.shared.array(shape=(blockDimX, blockDimY), dtype=float32)
        tileVT= cuda.shared.array(shape=(blockDimX, blockDimY), dtype=float32)
        tileS = cuda.shared.array(shape=(blockDimX,),           dtype=float32)
        
        # Number of chunks
        num_chunks = (k + blockDimX - 1) // blockDimX

        # Register accumulator
        accum = 0.0

        for chunk_idx in range(num_chunks):
            k_base = chunk_idx * blockDimX

            # Load from U => tileU[local_x, local_y]
            col_u = k_base + local_y
            if i < u.shape[0] and col_u < k:
                tileU[local_x, local_y] = u[i, col_u]
            else:
                tileU[local_x, local_y] = 0.0

            # Load from V^T => tileVT[local_x, local_y]
            row_vt = k_base + local_x
            if row_vt < vt.shape[0] and j < vt.shape[1]:
                tileVT[local_x, local_y] = vt[row_vt, j]
            else:
                tileVT[local_x, local_y] = 0.0

            # Load from s => tileS[local_x], only if local_y == 0
            if local_y == 0:
                if row_vt < s.shape[0]:
                    tileS[local_x] = s[row_vt]
                else:
                    tileS[local_x] = 0.0

            cuda.syncthreads()

            # Accumulate partial sums in registers
            for n in range(blockDimY):
                real_k = k_base + n
                if real_k < k:
                    accum += tileU[local_x, n] * tileS[local_x] * tileVT[local_x, n]

            cuda.syncthreads()

        C[i, j] = accum

    return reconstruct_svd_optimized
### END SOLUTION

In [ ]:
stream = cuda.stream()

results_optimized = []

for N in matrix_sizes:
    # Get precomputed components
    u = svd_components[N]['u']
    s = svd_components[N]['s'] 
    vt = svd_components[N]['vt']
    k = svd_components[N]['k']


    # Send data to device (on 'stream')
    u_dev = cuda.to_device(u, stream=stream)
    s_dev = cuda.to_device(s, stream=stream)
    vt_dev= cuda.to_device(vt, stream=stream)
    C_dev = cuda.device_array((N, N), dtype=np.float32, stream=stream)

    stream.synchronize()  # ensure transfers are done

    for block_size in block_configs:
        blockDimX, blockDimY = block_size
        gridX = math.ceil(N / blockDimX)
        gridY = math.ceil(N / blockDimY)
        grid = (gridX, gridY)

        kernel = get_optimized_kernel(block_size)

        kernel[grid, block_size, stream](u_dev, s_dev, vt_dev, C_dev, k)
        stream.synchronize()

        start_gpu = time.time()
        kernel[grid, block_size, stream](u_dev, s_dev, vt_dev, C_dev, k)
        gpu_time = time.time() - start_gpu

        stream.synchronize()

        C_gpu = C_dev.copy_to_host(stream=stream)
        stream.synchronize()

        # Compute speedup
        speedup = svd_components[N]['cpu_time'] / gpu_time

        results_optimized.append({
            'matrix_size': N,
            'block_size': block_size,
            'cpu_time': svd_components[N]['cpu_time'],
            'gpu_time': gpu_time,
            'speedup': speedup,
        })

        print(f"[N={N}] block={block_size} GPU={gpu_time:.5f}s CPU={cpu_time:.5f}s speedup={speedup:.2f}x")

    # Clean up
    del u_dev, s_dev, vt_dev, C_dev

In [ ]:
plot_results(results_optimized, 'Optimized Kernel')

In [ ]:
# Compare basic vs shared memory kernel performance
plt.figure(figsize=(15, 8))

# Get best times for each matrix size
best_times_basic = {}
best_times_shared = {}
best_times_optimized = {}
for N in matrix_sizes:
    basic_N = [r for r in results_basic if r['matrix_size'] == N]
    shared_N = [r for r in results_shared if r['matrix_size'] == N]
    optimized_N = [r for r in results_optimized if r['matrix_size'] == N]
    best_times_basic[N] = min(basic_N, key=lambda x: x['gpu_time'])
    best_times_shared[N] = min(shared_N, key=lambda x: x['gpu_time'])
    best_times_optimized[N] = min(optimized_N, key=lambda x: x['gpu_time'])

# Plot speedups
plt.subplot(1, 2, 1)
plt.plot(matrix_sizes, [best_times_basic[N]['speedup'] for N in matrix_sizes], 'b-o', label='Basic Kernel')
plt.plot(matrix_sizes, [best_times_shared[N]['speedup'] for N in matrix_sizes], 'r-o', label='Shared Memory Kernel')
plt.plot(matrix_sizes, [best_times_optimized[N]['speedup'] for N in matrix_sizes], 'g-o', label='Optimized Kernel')
plt.xlabel('Matrix Size')
plt.ylabel('Speedup vs CPU')
plt.title('Speedup Comparison')
plt.grid(True)
plt.legend()

# Plot execution times
plt.subplot(1, 2, 2)
plt.plot(matrix_sizes, [best_times_basic[N]['gpu_time'] for N in matrix_sizes], 'b-o', label='Basic Kernel')
plt.plot(matrix_sizes, [best_times_shared[N]['gpu_time'] for N in matrix_sizes], 'r-o', label='Shared Memory Kernel')
plt.plot(matrix_sizes, [best_times_optimized[N]['gpu_time'] for N in matrix_sizes], 'g-o', label='Optimized Kernel')
plt.xlabel('Matrix Size')
plt.ylabel('Execution Time (s)')
plt.title('Execution Time Comparison')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Print detailed comparison
print("\nDetailed Performance Comparison:")
print("-" * 80)
print(f"{'Matrix Size':^12} | {'Basic Time':^12} | {'Shared Time':^12} | {'Optimized Time':^12} | {'Improvement':^12} | {'Best Block Size':^20}")
print("-" * 80)

for N in matrix_sizes:
    basic = best_times_basic[N]
    shared = best_times_shared[N]
    optimized = best_times_optimized[N]
    improvement = (basic['gpu_time'] - shared['gpu_time']) / basic['gpu_time'] * 100
    
    print(f"{N:^12} | {basic['gpu_time']:^12.4f} | {shared['gpu_time']:^12.4f} | {improvement:^11.1f}% | {shared['block_size']}")

#### 5.3 NVIDIA Profiler

Benutze einen Performance Profiler von NVIDIA, um Bottlenecks in deinem Code zu identifizieren bzw. unterschiedliche Implementierungen (Blocks, Memory etc.) zu vergleichen. 

* Siehe Beispiel example_profiling_CUDA.ipynb
* [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) für das Profiling des Codes und die Inspektion der Ergebnisse (neuste Variante)
* [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview)
* [Nvidia Visual Profiler](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#visual)

> Du kannst NVIDIA Nsights Systems und den Nvidia Visual Profiler auf deinem PC installieren und die Leistungsergebnisse aus einer Remote-Instanz visualisieren, auch wenn du keine GPU an/in deinem PC hast. Dafür kannst du die ``*.qdrep`` Datei generieren und danach lokal laden.


Dokumentiere deine Analyse ggf. mit 1-2 Visualisierungen und beschreibe, welche Bottlenecks du gefunden bzw. entschärft hast.

<font color='blue'>Antwort hier eingeben inkl. Bild.</font>

### 6 Beschleunigte Rekonstruktion mehrerer Bilder
#### 6.1 Implementierung
Verwende einige der in bisher gelernten Konzepte, um mehrere Bilder gleichzeitig parallel zu rekonstruieren. Weshalb hast du welche Konzepte für deine Implementierung verwenden? Versuche die GPU konstant auszulasten und so auch die verschiedenen Engines der GPU parallel zu brauchen. Untersuche dies auch für grössere Inputs als die MRI-Bilder.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

<font color='blue'>Antwort hier eingeben</font>

#### 6.2 Analyse
Vergleiche den Speedup für deine parallele Implementierung im Vergleich zur seriellen Rekonstruktion einzelner Bilder. Analysiere und diskutiere in diesem Zusammenhang die Gesetze von Amdahl und Gustafson.

<font color='blue'>Antwort hier eingeben</font>

#### 6.3 Komponentendiagramm

Erstelle das Komponentendiagramm dieser Mini-Challenge für die Rekunstruktion mehrere Bilder mit einer GPU-Implementierung. Erläutere das Komponentendigramm in 3-4 Sätzen.


<font color='blue'>Antwort hier eingeben inkl. Bild(ern).</font>

### 7 Reflexion

Reflektiere die folgenden Themen indem du in 3-5 Sätzen begründest und anhand von Beispielen erklärst.

1: Was sind deiner Meinung nach die 3 wichtigsten Prinzipien bei der Beschleunigung von Code?

<font color='blue'>Antwort hier eingeben</font>

2: Welche Rechenarchitekturen der Flynnschen Taxonomie wurden in dieser Mini-Challenge wie verwendet?

<font color='blue'>Antwort hier eingeben</font>

3: Haben wir es in dieser Mini-Challenge hauptsächlich mit CPU- oder IO-Bound Problemen zu tun? Nenne Beispiele.

<font color='blue'>Antwort hier eingeben</font>

4: Wie könnte diese Anwendung in einem Producer-Consumer Design konzipiert werden?

<font color='blue'>Antwort hier eingeben</font>

5: Was sind die wichtigsten Grundlagen, um mehr Performance auf der GPU in dieser Mini-Challenge zu erreichen?

<font color='blue'>Antwort hier eingeben</font>

6: Reflektiere die Mini-Challenge. Was ist gut gelaufen? Wo gab es Probleme? Wo hast du mehr Zeit als geplant gebraucht? Was hast du dabei gelernt? Was hat dich überrascht? Was hättest du zusätzlich lernen wollen? Würdest du gewisse Fragestellungen anders formulieren? Wenn ja, wie?

<font color='blue'>Antwort hier eingeben</font>